# Предсказываем ответ пользователя с помощью RNN (GRU)

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time

import functools

### Подготовка данных
Данные хранятся в виде чатов:
* Одна строка - одна реплика
* Вопрос боту начинается с символа >
* Ответ бота начинается с символа <
* Чаты разделены строкой с текстом ===


In [2]:
# Загружаем набор чатов на которых будем обучаться
text = open('myMessages.txt', 'rb').read().decode(encoding='utf-8')
print ('Общее количество символов: {}'.format(len(text)))

# Составляем словарь символов
vocab = sorted(set(text))
print ('Уникальных символов: {}'.format(len(vocab)))

# Функции для преобразования текста в массив чисел и обратно
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

Общее количество символов: 13694221
Уникальных символов: 93


In [3]:
# Подгатавливаем обучающий датасет
seq_length = 100
examples_per_epoch = len(text)//seq_length
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [4]:
for input_example, target_example in  dataset.take(1):
  print ('Вход:  ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Выход:', repr(''.join(idx2char[target_example.numpy()])))

Instructions for updating:
Colocations handled automatically by placer.
Вход:   '> я вот что поставила\n===\n< ок\n< откуда лиса?\n> не згаю, бвла в телефоне. плохая?\n< нет хорошая\n> а '
Выход: ' я вот что поставила\n===\n< ок\n< откуда лиса?\n> не згаю, бвла в телефоне. плохая?\n< нет хорошая\n> а ч'


In [5]:
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

SHUFFLE_BUFFER_SIZE = 10000
dataset = dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

### Модель сети

In [27]:
# Размер используемого словаря
vocab_size = len(vocab)
# Размерности сети
embedding_dim = 256
rnn_units = 1024
rnn = functools.partial(tf.keras.layers.GRU, recurrent_activation='sigmoid')
# Для ускорения работы на GPU можно использовать rnn = tf.keras.layers.CuDNNGRU, но такая сеть не может быть потом 
# использована для работы на CPU.

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  return tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
      rnn(rnn_units,
          return_sequences=True,
          recurrent_initializer='glorot_uniform',
          stateful=True),
      tf.keras.layers.Dense(vocab_size)
  ])

In [28]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (64, None, 256)           23808     
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          3935232   
_________________________________________________________________
dense_2 (Dense)              (64, None, 93)            95325     
Total params: 4,054,365
Trainable params: 4,054,365
Non-trainable params: 0
_________________________________________________________________


In [13]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

### Обучение сети

In [14]:
# Настройка сохранения результатов
checkpoint_dir = 'training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [15]:
history = model.fit(dataset.repeat(), epochs=5, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/5
2138/2139 [============================>.] - ETA: 0s - loss: 1.7443WARNING:tensorflow:From /mnt/udata/anaconda/lib/python3.7/site-packages/tensorflow/python/keras/engine/network.py:1436: update_checkpoint_state (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.
2139/2139 [==============================] - 386s 181ms/step - loss: 1.7442
Epoch 2/5
2139/2139 [==============================] - 385s 180ms/step - loss: 1.4516
Epoch 3/5
2139/2139 [==============================] - 389s 182ms/step - loss: 1.4044
Epoch 4/5
2139/2139 [==============================] - 387s 181ms/step - loss: 1.3796
Epoch 5/5
2139/2139 [==============================] - 384s 179ms/step - loss: 1.3647


### Загрузка обученой модели из файла

In [32]:
# Загружаем модель из checkpoint'а, при этом используем batch_size размера 1, чтобы можно было использовать 
# модель в режиме чата
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (1, None, 256)            23808     
_________________________________________________________________
gru_6 (GRU)                  (1, None, 1024)           3935232   
_________________________________________________________________
dense_6 (Dense)              (1, None, 93)             95325     
Total params: 4,054,365
Trainable params: 4,054,365
Non-trainable params: 0
_________________________________________________________________


### Тестирование результатов работы

In [40]:
def generate_text(model, start_string, oneString, temperature):
  # Максимальное количество генерируемых символов
  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)

    if temperature > 0:
        predictions = predictions / temperature
    predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()

    input_eval = tf.expand_dims([predicted_id], 0)

    c = idx2char[predicted_id]
    text_generated.append(c)
    if c == '\n' and oneString:
        break

  return (start_string + ''.join(text_generated))

#### Автоматическое завершение чатов

In [112]:
print(generate_text(model, "===\n> куда завтра сходим?\n< в ", True, 0.001))
print(generate_text(model, "===\n> когда завтра сходим?\n< в ", True, 0.001))
print(generate_text(model, "===\n> где ты сейчас?\n< в ", True, 0.001))

===
> куда завтра сходим?
< в магазин

===
> когда завтра сходим?
< в 8

===
> где ты сейчас?
< в метро



#### Чат-бот

In [118]:
dialog = u"===\n"
while(True):
  rq = input("> ")
  if rq == '':
        break;
  dialog += "> " + rq + "\n< "
  
  fullAns = generate_text(model, start_string=dialog, temperature=0.3, oneString = True)
  shortAns = fullAns[len(dialog):]
  print("< " + shortAns)
  dialog = fullAns

> привет
< почему ты так подумал?

> когда домой идешь?
< да

> пока
< ну давай тогда позвоню

> 


#### Генератор чатов

In [116]:
print(generate_text(model, "===", False, 0.25))

===
> как дела?
< да ничего не понял
> ну так и не поняла
> ну как после поездки на работу?
< нет
> почему?
< ну вот так все понятно было
> ну вот так и не поедем
< ну вот ты скажешь что надо за ним поехать
< ну вот просто потом посмотрим
< надо было не поехать на дачу
< но не понятно как я так понял
> ну так и поедем
< ну вот приедем как с дачей в самолете
> ну вот скажи как ты сказал, что он не поедет на дачу
< ну там не получается подумать
> ну вот смотри, так и не понятно как его тебе не покупать
< ну почему не понятно
> ну вот посмотри как пойдем
< ну в смысле не помогло
> ну вот смотри, он же все против выгодно получить :(
< ну я понимаю что надо было покупать
> ну вот прямо сейчас выйдешь и поедем на дачу
< ну да, но не помню
> ну вот смотри, там все по сути он его не покупает
> ну и что?
< ну и в самом деле не получается
> ну вот смотри, почему ты не поедешь в какой-то проблему с кроватью по дороге после поездки в магазине на работу на дачу и все после работы в комнате поставит